In [18]:
%pip install -r ../Docs/requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [19]:
import torch
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is Apple MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is Apple MPS available? {torch.backends.mps.is_available()}")

# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

PyTorch version: 2.1.2
Is Apple MPS (Metal Performance Shader) built? True
Is Apple MPS available? True
Using device: mps


In [20]:
# Check if MPS is available, if yes, use MPS, else check for CUDA
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)
block_size = 8
batch_size = 4

mps


In [21]:
with open('../Datasets/Computers-the_machines_we_think_with.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))
chars = sorted(set(text))
print(chars)
print(len(chars))
vocabulary_size = len(chars)

483074
['\n', ' ', '!', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '£', '©', '°', '·', '×', 'à', 'á', 'ï', 'ó', 'ö', '÷', 'Ā', 'Č', '–', '—', '‘', '’', '“', '”', '…', '∨', '─', '○', '●']
109


In [22]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        29, 41, 39, 42, 47, 46, 31, 44, 45, 99, 46, 34, 31,  1, 39, 27, 29, 34,
        35, 40, 31, 45,  1, 49, 31,  1, 46, 34, 35, 40, 37,  1, 49, 35, 46, 34,
         0,  0,  0,  0,  0,  0,  0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10])


In [23]:
n = int(0.8*len (data))
train_data= data[:n]
val_data= data[n:]

In [24]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range (block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([1]) target is tensor(1)
when input is tensor([1, 1]) target is tensor(1)
when input is tensor([1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
when input is tensor([1, 1, 1, 1, 1, 1, 1, 1]) target is tensor(1)
